<a href="https://colab.research.google.com/github/vicentemf/Mercado-Financeiro/blob/main/Value_at_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  Value at Risk(VaR) é um conceito utilizado pelo setor de risco para quantificar o quanto o capital está exposto em determinado(s) investimento(s). Isto não impede que o investidor pessoa física também possa quantificar seu risco em carteira , este é o objetivo deste artigo.

  A idéia central do Var é quantificar a um determinado nível de confiança(geralmente 95% e 99%) o quanto se pode perder em determinado prazo(usarei neste artigo o diário) . 
  
  A literatura financeira menciona alguns modos para este cálculo,este artigo apresenta um exemplo do modelo de cálculo através da variância-covariância, que parte do princípio que os retornos dos investimentos são dados a partir de uma curva normal 

  Recomendo a palestra de Kenneth Abbott disponível no youtube para entendimento do conceito do cálculo.
  https://www.youtube.com/watch?v=92WaNz9mPeY

In [63]:
#biblitecas fundamentais para extração,visualização e análise de dados
!pip install yfinance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.stats import norm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Aqui pego as primeiras 2 ações que veio a mente,deixo claro que podemos utilizar quantas forem necessárias. Quanto as datas também estamos livres para escolher,entretanto,estatisticamente falando séries históricas longas trazem mais confiança a análise que séries históricas curtas .

In [64]:
carteira=["VALE3.SA","PETR4.SA"]
inicio="2010-01-01"
fim="2020-01-01"

In [65]:
#obtenção dos dados a partir do Yahoo Finance e criação de um DataFrame para visualização dos mesmos
cotacao=pd.DataFrame()
for i in carteira:
  cotacao[i]=yf.download(i,inicio,fim)["Adj Close"]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# Em uma análise, os dados nulos podem atrapalhar bastante,gosto de passar o comando abaixo para me certificar se tenho este tipo de dados em minha série histórica. O bom de usar a biblioteca Yahoo Finance é que dados nulos são bem difíceis de ocorrer.

In [66]:

cotacao.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2480 entries, 2010-01-04 to 2019-12-30
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   VALE3.SA  2480 non-null   float64
 1   PETR4.SA  2480 non-null   float64
dtypes: float64(2)
memory usage: 58.1 KB


#  Aqui temos um ponto muito importante,para fins de comparação devemos sempre normalizar os nossos dados para conseguirmos relacionar variáveis de diferentes ordens magnitudes. No artigo sobre "a recuperação dos bancos" neste mesmo repositório eu utilizo a normalização a 100 , aqui utilizei a variação logarítmica, recomendada para este tipo de caso na literatura financeira.

#   Em seguida atribui pesos para os papéis,40% em Vale3 e 60% em Petr4,proporção dada apenas para fins didáticos.


In [67]:

retorno=np.log(cotacao/cotacao.shift(1)).dropna()
pesos=np.array([.40,.60])
retorno["carteira"]=retorno.dot(pesos)

In [68]:
retorno.head(5)

,VALE3.SA,PETR4.SA,carteira
Date,,,
2010-01-05,0.009279,-0.008611,-0.001455
2010-01-06,0.020945,0.013423,0.016432
2010-01-07,0.004137,-0.009377,-0.003972
2010-01-08,0.009711,-0.005398,0.000646
2010-01-11,-0.002978,-0.003253,-0.003143


#  Além do retorno ponderado da carteira, precisamos da média e do desvio padrão 

In [69]:
media_retorno=retorno["carteira"].mean()
desvio=retorno["carteira"].std()
print(f"A média do retorno é = {media_retorno:.2%} ")
print(f"O desvio padrão é ={desvio:.2%} ")

A média do retorno é = 0.01% 
O desvio padrão é =2.30% 


# A funçao .ppf vai retorno a um determinado nível de confiança(1 - alpha) o valor crítico da nossa distribuição .Se desejar a um nível de confiança de 99% utilizar alpha =0.01 .

In [70]:
alpha=0.05
var=norm.ppf(alpha,media_retorno,desvio)
print(f"O valor do Value at Risk é de{var:.2%} para esta carteira")

O valor do Value at Risk é de-3.78% para esta carteira


In [71]:
capital=100000
perda_maxima=var * capital
print(f"Para o capital escolhido , ao nível de confiança de {1-alpha:.0%}, espera-se que o investidor não exceda a perda máxima de {perda_maxima:.2f} reais por dia" )

Para o capital escolhido , ao nível de confiança de 95%, espera-se que o investidor não exceda a perda máxima de -3782.21 reais por dia


# Com esta rotina podemos perceber a importância do setor de risco e porque da utilização do VaR para análises de investimentos . 
# É importante mencionar que toda base estatísica não apresenta certezas e sim probabilidades !